# Word2Vec (Negative Sampling)

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
np.__version__, torch.__version__

('2.2.0', '2.5.1+cu124')

In [3]:
import matplotlib
matplotlib.__version__

'3.10.0'

In [4]:
# Select the GPU with the most free memory
def get_free_gpu():
    free_mem = [torch.cuda.memory_reserved(i) for i in range(torch.cuda.device_count())]
    return free_mem.index(min(free_mem))

best_gpu = get_free_gpu()
torch.cuda.set_device(best_gpu)
print(f"Using GPU: {best_gpu}")

Using GPU: 0


In [5]:
# Set the device variable
device = torch.device(f'cuda:{best_gpu}' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

## 1. Load data

In [7]:
import nltk
nltk.download('reuters')
nltk.download('punkt_tab')

[nltk_data] Downloading package reuters to /home/jupyter-
[nltk_data]     st125469/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jupyter-
[nltk_data]     st125469/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [10]:
from nltk.corpus import reuters

In [11]:
reuters.words()

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', ...]

In [12]:
# corpus = ["apple banana fruit", "banana apple fruit", "banana fruit apple", "grape apple apple", 
#         "dog cat animal", "cat dog animal", "cat animal dog", "fish dog dog"]
corpus = reuters.sents()
len(corpus)

54716

In [8]:
# #1. tokenization
# corpus = [sent.split(" ") for sent in corpus]
# corpus

In [13]:
#2. numeralization
#find unique words
flatten = lambda l: [item for sublist in l for item in sublist]
#assign unique integer
vocabs = list(set(flatten(corpus))) #all the words we have in the system - <UNK>

In [14]:
#create handy mapping between integer and word
word2index = {v:idx for idx, v in enumerate(vocabs)}
word2index['dog']

26097

In [15]:
last_vocab_index = len(vocabs)
last_vocab_index

41599

In [16]:
vocabs.append('<UNK>')
word2index['<UNK>'] = last_vocab_index

In [17]:
index2word = {v:k for k, v in word2index.items()}
index2word[5]

'1p'

## 2. Prepare train data

In [18]:
#create pairs of center word, and outside word

def random_batch(batch_size, corpus, window_size=2):

    skipgrams = []

    #loop each corpus
    for doc in corpus:
        #look from the 2nd word until second last word
        for i in range(window_size, len(doc)-window_size):
            #center word
            center = word2index[doc[i]]
            #outside words = 2 words
            outside = tuple(word2index[doc[j]] for j in range(i - window_size, i + window_size + 1) if j != i)
            outside = (word2index[doc[i-2]], word2index[doc[i-1]], word2index[doc[i+1]], word2index[doc[i+2]])
            #for each of these two outside words, we gonna append to a list
            for each_out in outside:
                skipgrams.append([center, each_out])
                #center, outside1;   center, outside2
                
    random_index = np.random.choice(range(len(skipgrams)), batch_size, replace=False)
    
    inputs, labels = [], []
    for index in random_index:
        inputs.append([skipgrams[index][0]])
        labels.append([skipgrams[index][1]])
        
    return np.array(inputs), np.array(labels)
            
x, y = random_batch(2, corpus)

In [19]:
x.shape  #batch_size, 1

(2, 1)

In [20]:
x

array([[20762],
       [ 4281]])

In [21]:
y.shape  #batch_size 1

(2, 1)

## 3. Negative Sampling

### Unigram distribution

$$P(w)=U(w)^{3/4}/Z$$

In [22]:
z = 0.001

In [23]:
#count
from collections import Counter

word_count = Counter(flatten(corpus))
word_count

#get the total number of words
num_total_words = sum([c for w, c in word_count.items()])
num_total_words

1720917

In [24]:
vocabs[:100]

['traditional',
 'Flat',
 'FCL',
 'polypropylene',
 'REGIME',
 '1p',
 'fiduciary',
 'frame',
 'Newspapers',
 'shield',
 'rotated',
 'SFAS',
 'CREATION',
 '075',
 'RESIGNATION',
 '426',
 'Arabic',
 'Metallurgy',
 'Winthrop',
 'MACHINE',
 'Montgomery',
 'AMPLE',
 'pharmacist',
 'RAYMOND',
 'leases',
 'occupational',
 'PLACES',
 'TRANSWORLD',
 'Petrobras',
 'ARC',
 'Ruder',
 'ISTANBUL',
 'QUIX',
 'HOR',
 'BEL',
 'Appropriations',
 'extruded',
 'COUNTRY',
 'Passages',
 'Less',
 'drinks',
 'MANUFACTURING',
 'Held',
 'Club',
 'stocks',
 'SUPPLY',
 'writdown',
 'attempt',
 'DEFENSIVE',
 'dozen',
 'Seapride',
 'manoeuvred',
 'Les',
 '33p',
 'CRAFT',
 'instituting',
 'PLC',
 'priorities',
 'Murchison',
 'internally',
 'Albertans',
 'Minority',
 'UNLOAD',
 'kilowatt',
 '889',
 'Choice',
 'Duisburg',
 'Multnomah',
 'CUSHING',
 'resurrected',
 'mandates',
 'QUALIFIED',
 'contingent',
 'TRANSDUCER',
 'fifths',
 'ichi',
 'ORGANIZATION',
 '255',
 'relationship',
 'Quantities',
 'ESTIMATED',
 'predict

$$P(w)=U(w)^{3/4}/Z$$

In [25]:
unigram_table = []

for v in vocabs:
    uw = word_count[v] / num_total_words
    uw_alpha = int((uw ** 0.75) / z)
    unigram_table.extend([v] * uw_alpha)
    
Counter(unigram_table)

Counter({'.': 113,
         ',': 92,
         'the': 78,
         'of': 54,
         'to': 52,
         'in': 43,
         'said': 42,
         'and': 41,
         'a': 39,
         'mln': 32,
         'vs': 27,
         '-': 26,
         'for': 25,
         "'": 23,
         'dlrs': 23,
         'The': 22,
         '1': 21,
         '000': 21,
         ';': 19,
         'pct': 19,
         'it': 19,
         's': 19,
         'lt': 18,
         'on': 18,
         '&': 18,
         'cts': 17,
         'from': 17,
         'is': 17,
         '>': 16,
         'its': 16,
         'that': 16,
         'S': 15,
         'by': 15,
         'U': 15,
         '"': 15,
         'at': 15,
         '2': 15,
         'with': 14,
         'will': 14,
         'be': 14,
         'year': 14,
         'billion': 13,
         'was': 13,
         '3': 12,
         '5': 11,
         'would': 11,
         'company': 11,
         'not': 11,
         '4': 11,
         'as': 11,
         'loss': 11,
       

## 4. Model

$$\mathbf{J}_{\text{neg-sample}}(\mathbf{v}_c,o,\mathbf{U})=-\log(\sigma(\mathbf{u}_o^T\mathbf{v}_c))-\sum_{k=1}^K\log(\sigma(-\mathbf{u}_k^T\mathbf{v}_c))$$

In [26]:
def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index['<UNK>'], seq))
    return torch.LongTensor(idxs)

In [27]:
import random

def negative_sampling(targets, unigram_table, k):
    batch_size = targets.shape[0]
    neg_samples = []
    for i in range(batch_size):  #(1, k)
        target_index = targets[i].item()
        nsample      = []
        while (len(nsample) < k):
            neg = random.choice(unigram_table)
            if word2index[neg] == target_index:
                continue
            nsample.append(neg)
        neg_samples.append(prepare_sequence(nsample, word2index).reshape(1, -1))
        
    return torch.cat(neg_samples) #batch_size, k

In [28]:
batch_size = 2
x, y = random_batch(batch_size, corpus)
x_tensor = torch.LongTensor(x)
y_tensor = torch.LongTensor(y)

In [29]:
k = 5
neg_samples = negative_sampling(y_tensor, unigram_table, k)

In [30]:
y_tensor[1]

tensor([34544])

In [31]:
neg_samples[1]

tensor([ 3738,  6712, 17584, 40803, 14371])

$$\mathbf{J}_{\text{neg-sample}}(\mathbf{v}_c,o,\mathbf{U})=-\log(\sigma(\mathbf{u}_o^T\mathbf{v}_c))-\sum_{k=1}^K\log(\sigma(-\mathbf{u}_k^T\mathbf{v}_c))$$

In [33]:
class SkipgramNeg(nn.Module):
    
    def __init__(self, voc_size, emb_size, word2index):
        super(SkipgramNeg, self).__init__()
        self.embedding_center  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside = nn.Embedding(voc_size, emb_size)
        self.logsigmoid        = nn.LogSigmoid()
        self.word2index        = word2index
    
    def forward(self, center, outside, negative):
        #center, outside:  (bs, 1)
        #negative       :  (bs, k)
        
        center_embed   = self.embedding_center(center) #(bs, 1, emb_size)
        outside_embed  = self.embedding_outside(outside) #(bs, 1, emb_size)
        negative_embed = self.embedding_outside(negative) #(bs, k, emb_size)
        
        uovc           = outside_embed.bmm(center_embed.transpose(1, 2)).squeeze(2) #(bs, 1)
        ukvc           = -negative_embed.bmm(center_embed.transpose(1, 2)).squeeze(2) #(bs, k)
        ukvc_sum       = torch.sum(ukvc, 1).reshape(-1, 1) #(bs, 1)
        
        loss           = self.logsigmoid(uovc) + self.logsigmoid(ukvc_sum)
        
        return -torch.mean(loss)

    def get_embed(self, word):
      word2index = self.word2index
      try:
        index = word2index[word]
      except:
        index = word2index['<UNK>']

      word = torch.LongTensor([word2index[word]]).to(device)

      embed_c = self.embedding_center(word)
      embed_o = self.embedding_outside(word)
      embed   = (embed_c + embed_o) / 2
    
      return embed[0][0].item(), embed[0][1].item()

In [36]:
#test your model
emb_size = 2
voc_size = len(vocabs)
model = SkipgramNeg(voc_size, emb_size, word2index)

In [37]:
loss = model(x_tensor, y_tensor, neg_samples)

In [38]:
loss

tensor(2.5343, grad_fn=<NegBackward0>)

## 5. Training

In [39]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [42]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_time, elapsed_mins, elapsed_secs

In [43]:
import time

num_epochs = 100
total_time = 0
start_time = time.time()

for epoch in range(num_epochs):
    
    #get batch
    input_batch, label_batch = random_batch(batch_size, corpus)
    input_tensor = torch.LongTensor(input_batch)
    label_tensor = torch.LongTensor(label_batch)
    
    #predict
    neg_samples = negative_sampling(label_tensor, unigram_table, k)
    loss = model(input_tensor, label_tensor, neg_samples)
    
    #backprogate
    optimizer.zero_grad()
    loss.backward()
    
    #update alpha
    optimizer.step()
    
    #print the loss
    if (epoch + 1) % 10 == 0:
        end_time = time.time()
        total, epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        total_time += total
        print(f"Epoch {epoch+1:6.0f} | Loss: {loss:2.6f} | time: {epoch_mins}m {epoch_secs}s")
        start_time = time.time()

Epoch     10 | Loss: 2.015184 | time: 2m 3s
Epoch     20 | Loss: 1.985616 | time: 2m 4s
Epoch     30 | Loss: 1.890364 | time: 2m 2s
Epoch     40 | Loss: 2.655458 | time: 2m 3s
Epoch     50 | Loss: 3.335845 | time: 2m 7s
Epoch     60 | Loss: 0.228589 | time: 2m 4s
Epoch     70 | Loss: 1.044566 | time: 2m 5s
Epoch     80 | Loss: 1.309772 | time: 2m 16s
Epoch     90 | Loss: 1.105003 | time: 2m 14s
Epoch    100 | Loss: 1.163552 | time: 2m 17s


In [44]:
# print total train loss and total training time
print(f"Total train loss: {loss:.6f}")
print(f"Total training time: {total_time:.2f} seconds")

Total train loss: 1.163552
Total training time: 1278.51 seconds


## 6. Plot the embeddings

In [45]:
vocabs[:10]

['traditional',
 'Flat',
 'FCL',
 'polypropylene',
 'REGIME',
 '1p',
 'fiduciary',
 'frame',
 'Newspapers',
 'shield']

In [47]:
frame = torch.LongTensor([word2index['frame']])
frame

tensor([7])

In [48]:
frame_embed_c = model.embedding_center(frame)
frame_embed_o = model.embedding_outside(frame)
frame_embed   = (frame_embed_c + frame_embed_o) / 2
frame_embed

tensor([[-1.4728,  1.0519]], grad_fn=<DivBackward0>)

In [49]:
frame_embed_o

tensor([[-2.9622,  1.6424]], grad_fn=<EmbeddingBackward0>)

In [50]:
# def get_embed(word):
#     try:
#         index = word2index[word]
#     except:
#         index = word2index['<UNK>']
        
#     word = torch.LongTensor([word2index[word]])
    
#     embed_c = model.embedding_center(word)
#     embed_o = model.embedding_outside(word)
#     embed   = (embed_c + embed_o) / 2
    
#     return embed[0][0].item(), embed[0][1].item()

In [51]:
# get_embed('fruit')

In [52]:
# get_embed('cat')

In [53]:
# get_embed('dog')

In [54]:
# get_embed('banana')

In [55]:
# plt.figure(figsize=(6, 3))
# for i, word in enumerate(vocabs):
#     x, y = get_embed(word)
#     plt.scatter(x, y)
#     plt.annotate(word, xy=(x, y), xytext=(5, 2), textcoords='offset points')
# plt.show()

## 7. Save Model

In [56]:
torch.save(model.state_dict(), 'app/code/models/skipgram-neg.pt')

In [57]:
import pickle

skipgram_neg_args = {
    'word2index': word2index,
    'voc_size': voc_size,
    'emb_size': emb_size
}

pickle.dump(skipgram_neg_args, open('app/code/models/skipgrams-neg.pkl', 'wb'))

In [60]:
load_skipgram_neg_args = pickle.load(open('app/code/models/skipgrams-neg.pkl', 'rb'))
load_model = SkipgramNeg(**load_skipgram_neg_args).to(device)
load_model.load_state_dict(torch.load('app/code/models/skipgram-neg.pt'))

/tmp/ipykernel_925156/433674238.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_model.load_state_dict(torch.load('app/code/models/skipgram-neg.pt'))


<All keys matched successfully>

In [61]:
load_model.get_embed('frame')

(-1.47282075881958, 1.0519459247589111)

## 7. Cosine similarity

In [62]:
# banana = get_embed('banana')
# banana

In [63]:
# fruit = get_embed('fruit')
# fruit

In [64]:
# cat = get_embed('cat')
# cat

In [65]:
# np.array(banana) @ np.array(cat)

In [66]:
# #more formally is to divide by its norm
# def cosine_similarity(A, B):
#     dot_product = np.dot(A, B)
#     norm_a = np.linalg.norm(A)
#     norm_b = np.linalg.norm(B)
#     similarity = dot_product / (norm_a * norm_b)
#     return similarity

# print(cosine_similarity(np.array(banana), np.array(cat)))
# print(cosine_similarity(np.array(banana), np.array(fruit)))